Optimise CNOT in quantum system

In [1]:
#QuTiP
import qutip.logging_utils as logging
logger = logging.get_logger()
#Local
import qsoconfig, qso, qsorun



Configuration

In [ ]:
# Create the objects for the configuration
optim = qsoconfig.gen_config_objects()
cfg = optim.config

tc = optim.termination_conditions
fid_comp = dyn.fid_computer

In [ ]:
# Configure the quantum system
dyn = optim.dynamics

